### Paralogs genes

#### 1.1 Including libraries

In [4]:
from Bio.Blast import NCBIWWW
from Bio.Blast import Applications
from Bio.Blast.Applications import NcbiblastxCommandline
import gzip 
import io
import math
import matplotlib.pyplot as plt
from ftplib import FTP
import os
import numpy as np
import pandas as pd
import patoolib
import random
import re
import seaborn as sns
import scipy as sc
import string
from tqdm import tqdm
import datetime
import xml.etree.ElementTree as ET
%matplotlib inline

#### 1.2  Data from NCBI for Blastp

In [48]:
def data_for_blastp(bacterium):
    # diving to database
    ftp = FTP('ftp.ncbi.nlm.nih.gov')
    ftp.login()
    ftp.cwd('genomes/refseq/bacteria')
    ftp.cwd(bacterium)
    
    domain_list = ['representative','reference','latest_assembly_versions']
    for domain in domain_list:
        if domain in ftp.nlst():
            ftp.cwd(domain)
            break
            
    current_strain = ftp.nlst()
    ftp.cwd(current_strain[0])
     
    namefile = current_strain[0] + '_protein.faa.gz' # file, which i want
    out = "C://idea_projects_my//my_file.gz" # place on my disk
    
    with open(out,'wb') as f:
        ftp.retrbinary('RETR ' + namefile,f.write)
        f_read = gzip.open(out,'rb')
    
    os.mkdir('C:\\idea_projects_my\\scherichia')
    patoolib.extract_archive(out,outdir = 'C:\\idea_projects_my\\scherichia')  
    os.listdir('C:\\idea_projects_my\\scherichia')
    new_out = 'C:\\idea_projects_my\\scherichia'
    for unarchived_file in os.listdir('C:\\idea_projects_my\\scherichia'):
        with open('C:\\idea_projects_my\\scherichia' + '\\' + unarchived_file, 'r') as myfile:
            data = myfile.read()
    #remove directories       
    os.remove('C:\\idea_projects_my\\scherichia\\my_file')
    os.rmdir('C:\\idea_projects_my\\scherichia')
    
    
    return data

In [12]:
def splitting(data):
    names_of_genes = data.split(sep = '>WP_')
    
    return names_of_genes

#### 1.3 Parse XML file

In [ ]:
def Parse_XML_and_holder(xml_current_file,gene_1):
    name_of_gene,length,identity,e_value,gen = list(),list(),list(),list(),list()
    with open('docu.xml','w') as file:
        file.write(str(xml_current_file))
        
    root = ET.parse('docu.xml').getroot()
    for subtags in root.iter('Iteration_hits'):
        for subsubtags in subtags:
            for objects in subsubtags:
                if(objects.tag == 'Hit_def'):
                    name_of_gene_cur = objects.text
                if(objects.tag == 'Hit_len'):
                    length_cur = int(objects.text)
                if(objects.tag == 'Hit_hsps'):
                    for tags in objects[0]:
                        if(tags.tag == 'Hsp_evalue'):
                            e_value_cur = float(tags.text)
                        if(tags.tag == 'Hsp_identity'):
                            identity_cur = int(tags.text)
                            if(identity_cur/length_cur > 0.5 and identity_cur/length_cur != 1):
                                gen.append(gene_1)
                                name_of_gene.append(name_of_gene_cur)
                                length.append(length_cur)
                                e_value.append(e_value_cur)
                                identity.append(float(identity_cur/length_cur))
                        
    return name_of_gene,length,identity,e_value,gen 

#### 1.4 writing genes for files and Blastp

In [49]:
def base_program(names_of_genes,bacterium):
    
    gen,gene_2,length,identity,e_value,bact = list(),list(),list(),list(),list(),list()
    #ways for genes
    out_sample = 'C://idea_projects_my//Escherichia_sample'
    out_database = 'C://idea_projects_my//Escherichia_database'
    
    for i, sample in tqdm(enumerate(names_of_genes)):
        database = names_of_genes[i:]
        try:
            with open(out_sample,'w')as file:
                file.write(str(sample))
            with open(out_database,'w')as file:
                file.write("".join(map(lambda x: ">WP_" + x, database)))
        except(FileNotFoundError,OSError):
            print('problems with directories')
            continue
        
        gene_1 = "WP_" + str(sample)[: str(sample).find(" ")]
        
        !makeblastdb -in C://idea_projects_my//Escherichia_database -dbtype prot -out C://idea_projects_my//Escherichia_database      
        
        result = NcbiblastxCommandline(cmd='blastp',num_threads = 4,query= out_sample,
                              db=out_database,evalue=1e-9,outfmt = 5)
        
        try:
            #out, err = result()
            output_of_xml = Parse_XML_and_holder(result()[0],gene_1)
            gene_2.extend(output_of_xml[0])
            length.extend(output_of_xml[1])
            identity.extend(output_of_xml[2])
            e_value.extend(output_of_xml[3])
            gen.extend(output_of_xml[4])
            
        except:
            print("oops" + str(i))
            continue
            
        
    return gene_2,length,identity,e_value,bacterium,gen

#### 2. Searching placements of data for each gene

In [ ]:
def data_for_placements(bacterium):
    # diving to database
    ftp = FTP('ftp.ncbi.nlm.nih.gov')
    ftp.login()
    ftp.cwd('genomes/refseq/bacteria')
    ftp.cwd(bacterium)
    
    domain_list = ['representative','reference','latest_assembly_versions']
    for domain in domain_list:
        if domain in ftp.nlst():
            ftp.cwd(domain)
            break
            
    current_strain = ftp.nlst()
    ftp.cwd(current_strain[0])
     
    namefile = current_strain[0] + '_translated_cds.faa.gz' # file, which i want
    out = "C://idea_projects_my//my_file.gz" # place on my disk
    
    with open(out,'wb') as f:
        ftp.retrbinary('RETR ' + namefile,f.write)
        f_read = gzip.open(out,'rb')
    
    os.mkdir('C:\\idea_projects_my\\scherichia')
    patoolib.extract_archive(out,outdir = 'C:\\idea_projects_my\\scherichia')  
    os.listdir('C:\\idea_projects_my\\scherichia')
    new_out = 'C:\\idea_projects_my\\scherichia'
    for unarchived_file in os.listdir('C:\\idea_projects_my\\scherichia'):
        with open('C:\\idea_projects_my\\scherichia' + '\\' + unarchived_file, 'r') as myfile:
            data = myfile.read()
    #remove directories       
    os.remove('C:\\idea_projects_my\\scherichia\\my_file')
    os.rmdir('C:\\idea_projects_my\\scherichia')
    
    
    return data

In [ ]:
def splitting(data):
    genes = data.split(sep = '>')
    genes = genes[1:]
    return genes

In [ ]:
def information_of_gene(gene,genes):
    for element in genes:
        if(gene in element):
            return element

In [ ]:
def deriving_number(query):
    if('complement' in query):
        s = re.findall(r'complement\D\w+',query)
        number = int(re.findall(r'\w+',str(s))[1])
    else:
        s = re.findall(r'location=\w+',query)
        number = int(re.findall(r'\w+',str(s))[1])
        

#### 3.Origin and Terminator

In [ ]:
from __future__ import division, print_function
import numpy as np

def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising',
                 kpsh=False, valley=False, show=False, ax=None):
     x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
    # find indices of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan-1, indnan+1))), invert=True)]
    # first and last values of x cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size-1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind]-x[ind-1], x[ind]-x[ind+1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                # keep peaks with the same height if kpsh is True
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                    & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0  # Keep current peak
        # remove the small peaks and sort back the indices by their occurrence
        ind = np.sort(ind[~idel])

    if show:
        if indnan.size:
            x[indnan] = np.nan
        if valley:
            x = -x
        #_plot(x, mph, mpd, threshold, edge, valley, ax, ind)

    return ind


def _plot(x, mph, mpd, threshold, edge, valley, ax, ind):
    """Plot results of the detect_peaks function, see its help"""
    try:
        import matplotlib.pyplot as plt
    except ImportError:
        print('matplotlib is not available.')
    else:
        if ax is None:
            _, ax = plt.subplots(1, 1, figsize=(8, 4))

        ax.plot(x, 'b', lw=1)
        if ind.size:
            label = 'valley' if valley else 'peak'
            label = label + 's' if ind.size > 1 else label
            ax.plot(ind, x[ind], '+', mfc=None, mec='r', mew=2, ms=8,
                    label='%d %s' % (ind.size, label))
            

In [ ]:
def calc_GC_skew(seq):
    """Calculate the GC-skew for a given sequence"""
    

    values = []
    window = int(0.000005 * len(seq))
    # window = 25
    for i in range(0, len(seq), window):
        s = seq[i: i + window]
        g = s.count('G') + s.count('g')
        c = s.count('C') + s.count('c')
        skew = (g-c)#/float(g+c)
        values.append(skew)

    
    return values, window


def smoothListGaussian(myarray, degree=5000):
    """Smooth out the cumulative skew values for re-plotting"""
    myarray  = np.pad(myarray, (degree-1, degree-1), mode='edge')
    window   = degree*2-1
    weight   = np.arange(-degree+1, degree)/window
    weight   = np.exp(-(16*weight**2))
    weight  /= sum(weight)
    smoothed = np.convolve(myarray, weight, mode='valid')
    return smoothed


def plot_skew(skewArray):
    """Plots a skew diagram and returns the respective cumulative values"""

    skewCumValues = np.cumsum(skewArray)
    #fig = plt.figure()
    #ax1 = fig.add_subplot(211)
    #ax2 = fig.add_subplot(212)

    ## ax1.plot(range(len(skewArray)), skewArray, 'r', color='blue')  # noise values
    #ax1.plot(range(len(skewArray)), skewCumValues, 'r', color='red') # cumulative values

    smoothedY = smoothListGaussian(skewCumValues)
    #ax2.plot(range(len(smoothedY)), smoothedY, 'r', color='black')   # smooth values

    #fig.savefig('SkewPlot.png')

    
    return(skewCumValues, smoothedY)


def identifyPeaks(data, stepsize):
    """Identify peaks/valleys (global min/max) from the smoothed data"""
    ind = detect_peaks(data, mpd=stepsize ,valley=True , show=True)
    return (ind)

def identifyPeaks_max(data,stepsize):
    ind = detect_peaks(data,mpd = stepsize,valley = False,show = True)
    return(ind)

def identifyPeaks_min(data,stepsize):
    ind = detect_peaks(data,mpd = stepsize,valley = True,show = True)
    return (ind)

In [ ]:
def searching_of_maxim_and_minim(bacterium):
    
    """
    >>> searching_of_maxim_and_minim('Acaryochloris_marina')
    (1129212, 3134586, 8466864, 42)
    
    >>> searching_of_maxim_and_minim('Acholeplasma_laidlawii')
    (723331, 0, 1515749, 7)
    
    >>> searching_of_maxim_and_minim('Anaerolinea_thermophila')
    (1708704, 3289534, 3576582, 17)
    
    
    
    
    """
    
    
    ftp = FTP('ftp.ncbi.nlm.nih.gov')
    ftp.login()
    ftp.cwd('genomes/refseq/bacteria')
    ftp.cwd(bacterium)
    
    domain_list = ['representative','reference','latest_assembly_versions']
    for domain in domain_list:
        if domain in ftp.nlst():
            ftp.cwd(domain)
            break
            
    current_strain = ftp.nlst()
    ftp.cwd(current_strain[0])
     
    namefile = current_strain[0] + '_genomic.fna.gz' # file, which i want
    out = "C://idea_projects_my//my_file.gz" # plac
    with open(out,'wb') as f:
        ftp.retrbinary('RETR ' + namefile,f.write)
        f_read = gzip.open(out,'rb')
    
    os.mkdir('C:\\idea_projects_my\\scherichia')
    patoolib.extract_archive(out,outdir = 'C:\\idea_projects_my\\scherichia')  
    os.listdir('C:\\idea_projects_my\\scherichia')
    new_out = 'C:\\idea_projects_my\\scherichia'
    for unarchived_file in os.listdir('C:\\idea_projects_my\\scherichia'):
        with open('C:\\idea_projects_my\\scherichia' + '\\' + unarchived_file, 'r') as myfile:
            data = myfile.read()
    #remove directories       
    os.remove('C:\\idea_projects_my\\scherichia\\my_file')
    os.rmdir('C:\\idea_projects_my\\scherichia')
    
    
            
    counter = 0
    for element in data:
        counter+=1
        if element in ['A','T']:
            break
    data = data[counter + 1:]
    
    length_of_sequence = len(data)
    coefficient = int(len(data)/200000)
    
    skewArray,window = calc_GC_skew(data)
    skewCumValues,smoothValues = plot_skew(skewArray)
    stepsize = 500
    identifiedPeaks_maxim = identifyPeaks_max(smoothValues,stepsize)
    identifiedPeaks_minim = identifyPeaks_min(smoothValues,stepsize)   

    prev_element = 0
    output = 0

    for element in identifiedPeaks_minim:
        if smoothValues[element] < smoothValues[prev_element]:
            output = element
            prev_element = element
    
    our_minimum = output

    prev_element = 0
    output = 0

    for element in identifiedPeaks_maxim:
        if smoothValues[element] > smoothValues[prev_element]:
            output = element
            prev_element = element
    
    our_maximum = output
    
    our_maximum *= coefficient
    our_minimum *= coefficient
    
    return our_maximum,our_minimum,length_of_sequence,coefficient
            
    

### 4. Base code

In [ ]:
df_sum = pd.DataFrame()
df_bac = pd.read_csv('olga_november.csv') # names of our bacteria
bacteria = list(df_bac['bact'])

In [ ]:
for bacterium in tqdm(bacteria):
    
    data = data_for_blastp(bacterium)
    names_of_genes = splitting(data)
    final = base_program(names_of_genes,bacterium)
    gene_1 = final[5]
    f = lambda x: str(x)[:str(x).find(" ")]
    gene_2 = [f(x) for x in final[0]]
    
    gene_1 = [x[3:] for x in gene_1]
    gene_2 = [x[3:] for x in gene_2]
    
    location_of_gene_1 = []
    location_of_gene_2 = []
    data = data_for_placements(bacterium)
    res = splitting(data)
    for gene in gene_1:
        query = information_of_gene(gene,res)
        try:
            location_of_gene_1.append(deriving_number(query))
        except:
            location_of_gene_1.append(-1)
            continue
    for gene in gene_2:
        query = information_of_gene(gene,res)
        try:
            location_of_gene_2.append(deriving_number(query))
        except:
            location_of_gene_2.append(-1)
            continue
            
            
    list_maxim = list()
    list_minim = list()
    length_of_sequence = list()
    result = searching_of_maxim_and_minim(bacterium)
    list_maxim.append(result[0])
    list_minim.append(result[1])
    length_of_sequence.append(result[2])
    data = {"bact":final[4],"gene_1": gene_1, "gene_2" : gene_2,"loc_1":location_of_gene_1,"loc_2":location_of_gene_2,
           "identity":final[2],"length":final[1],"e_value":final[3],"origin":list_minim[0],"terminus":list_maxim[0]}
    df = pd.DataFrame(data)
    df_sum = pd.concat([df,df_sum])
    df_sum.to_csv('olga_january.csv')